# BNP Paribas Web Scraper for RAG Pipeline

**This notebook scrapes data from BNP Paribas website and prepares it for RAG (Retrieval-Augmented Generation).**

**Features:**
- Scrapes content from multiple categories
- Cleans and structures data
- Exports in multiple formats (JSON)
- Ready for vector databases and RAG systems

In [3]:
import sys
import scipy
import numpy as np
import matplotlib
import pandas as pd
import sklearn
from pandas.plotting import scatter_matrix
from matplotlib import pyplot


bnp_link = pd.read_csv('/content/BNPParibas_Top10_Chatbot_Links.csv')
print(bnp_link.shape)
bnp_link.head(20)


(10, 3)


,Catégorie,URL,Description
0,Accueil / Vue d’ensemble,https://mabanque.bnpparibas/,Présentation générale et liens vers toutes les...
1,Comptes & Formules,https://mabanque.bnpparibas/fr/devenir-client/...,"Détails sur les comptes BNP Paribas, formules ..."
2,Épargne & Livrets,https://mabanque.bnpparibas/fr/epargner/compte...,"Produits d’épargne : Livret A, LDDS, PEL, CEL,..."
3,Crédits & Financement,https://mabanque.bnpparibas/fr/vos-besoins/vos...,"Solutions de financement : prêts personnels, a..."
4,Cartes bancaires,https://mabanque.bnpparibas/fr/gerer/services-...,"Cartes Visa Classic, Premier, Infinite : avant..."
5,Bourse & Placement,https://mabanque.bnpparibas/fr/bourse/nos-offr...,Informations sur la bourse et les placements f...
6,Assurances,https://mabanque.bnpparibas/fr/assurer,"Assurances habitation, auto, santé, prévoyance."
7,FAQ / Aide & Services,https://mabanque.bnpparibas/fr/ma-banque-et-mo...,Page FAQ et services d’accompagnement client.
8,Tarifs & Conditions,https://mabanque.bnpparibas/fr/notre-offre/tarifs,Grille tarifaire BNP Paribas et conditions gén...
9,Sécurité / Connexion,https://mabanque.bnpparibas/fr/securite,Conseils sécurité et prévention des fraudes ba...


In [4]:
bnp_link_df = bnp_link[bnp_link['Catégorie'] != 'Assurances']
display(bnp_link_df)

,Catégorie,URL,Description
0,Accueil / Vue d’ensemble,https://mabanque.bnpparibas/,Présentation générale et liens vers toutes les...
1,Comptes & Formules,https://mabanque.bnpparibas/fr/devenir-client/...,"Détails sur les comptes BNP Paribas, formules ..."
2,Épargne & Livrets,https://mabanque.bnpparibas/fr/epargner/compte...,"Produits d’épargne : Livret A, LDDS, PEL, CEL,..."
3,Crédits & Financement,https://mabanque.bnpparibas/fr/vos-besoins/vos...,"Solutions de financement : prêts personnels, a..."
4,Cartes bancaires,https://mabanque.bnpparibas/fr/gerer/services-...,"Cartes Visa Classic, Premier, Infinite : avant..."
5,Bourse & Placement,https://mabanque.bnpparibas/fr/bourse/nos-offr...,Informations sur la bourse et les placements f...
7,FAQ / Aide & Services,https://mabanque.bnpparibas/fr/ma-banque-et-mo...,Page FAQ et services d’accompagnement client.
8,Tarifs & Conditions,https://mabanque.bnpparibas/fr/notre-offre/tarifs,Grille tarifaire BNP Paribas et conditions gén...
9,Sécurité / Connexion,https://mabanque.bnpparibas/fr/securite,Conseils sécurité et prévention des fraudes ba...


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
from pathlib import Path
import re
from IPython.display import display


# Configuration
OUTPUT_DIR = "bnp_rag_documents"
DELAY_BETWEEN_REQUESTS = 2.0  # seconds

# Create output directory
Path(OUTPUT_DIR).mkdir(exist_ok=True)

# Headers for requests
HEADERS = {
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
}

# Store all documents
documents = []

In [6]:
def clean_text(text):
    """Clean extracted text"""
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

# %%
def scrape_page(url, category, description, doc_id):
    """Scrape a single page and extract relevant content"""
    try:
        print(f"Scraping: {category}")
        response = requests.get(url, headers=HEADERS, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Remove script and style elements
        for element in soup(['script', 'style', 'nav', 'footer', 'header']):
            element.decompose()

        # Extract title
        title = soup.find('h1')
        title_text = title.get_text(strip=True) if title else category

        # Extract main content
        content_selectors = [
            'main',
            'article',
            '[role="main"]',
            '.content',
            '.main-content',
            '#content'
        ]

        content = None
        for selector in content_selectors:
            content = soup.select_one(selector)
            if content:
                break

        if not content:
            content = soup.find('body')

        # Extract paragraphs and headings
        text_elements = []
        if content:
            for elem in content.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'li']):
                text = clean_text(elem.get_text())
                if len(text) > 20:
                    text_elements.append(text)

        full_text = ' '.join(text_elements)

        # Create document structure
        document = {
            'id': f"doc_{doc_id}",
            'category': category,
            'title': title_text,
            'url': url,
            'description': description,
            'content': full_text,
            'word_count': len(full_text.split()),
            'scraped_at': time.strftime('%Y-%m-%d %H:%M:%S')
        }

        print(f"   ✓ Extracted {len(full_text.split())} words\n")
        return document

    except requests.exceptions.RequestException as e:
        print(f"   ✗ Error: {e}\n")
        return {
            'id': f"doc_{doc_id}",
            'category': category,
            'title': category,
            'url': url,
            'description': description,
            'content': f"Error: Could not scrape this page. {str(e)}",
            'word_count': 0,
            'scraped_at': time.strftime('%Y-%m-%d %H:%M:%S'),
            'error': str(e)
        }

print(f"Starting scraping process...\n")

Starting scraping process...



In [7]:
# Scrape each URL
for idx, row in bnp_link_df.iterrows():
    category = row['Catégorie']
    url = row['URL']
    description = row['Description']

    document = scrape_page(url, category, description, idx)
    documents.append(document)

    # Be polite - add delay between requests
    if idx < len(bnp_link_df) - 1:
        time.sleep(DELAY_BETWEEN_REQUESTS)




print(f"{'='*60}")
print(f"✅ Scraping complete! {len(documents)} documents collected.")
print(f"{'='*60}\n")

Scraping: Accueil / Vue d’ensemble
   ✓ Extracted 1865 words

Scraping: Comptes & Formules
   ✗ Error: 404 Client Error: Not Found for url: https://mabanque.bnpparibas/fr/devenir-client/compte-bancaire

Scraping: Épargne & Livrets
   ✓ Extracted 170 words

Scraping: Crédits & Financement
   ✓ Extracted 14 words

Scraping: Cartes bancaires
   ✗ Error: 404 Client Error: Not Found for url: https://mabanque.bnpparibas/fr/gerer/services-bancaires/cartes

Scraping: Bourse & Placement
   ✓ Extracted 567 words

Scraping: FAQ / Aide & Services
   ✓ Extracted 2449 words

Scraping: Tarifs & Conditions
   ✗ Error: 404 Client Error: Not Found for url: https://mabanque.bnpparibas/fr/notre-offre/tarifs

Scraping: Sécurité / Connexion
   ✗ Error: 404 Client Error: Not Found for url: https://mabanque.bnpparibas/fr/securite

✅ Scraping complete! 9 documents collected.



In [8]:
documents

[{'id': 'doc_0',
  'category': 'Accueil / Vue d’ensemble',
  'title': 'BNP PARIBAS BANQUE ET ASSURANCES',
  'url': 'https://mabanque.bnpparibas/',
  'description': 'Présentation générale et liens vers toutes les offres.',
  'content': 'BNP PARIBAS BANQUE ET ASSURANCES Jusqu’à 270 € d’avantages offerts (1) Pour toute première ouverture d\'un compte BNP Paribas et d\'une carte Visa Premier avec l\'offre Esprit Libre Référence Soit 80 € offerts + 1 an de gratuité aux services essentiels Esprit Libre. Leader européen des services bancaires (2) Leader européen des services bancaires (2) Prix du meilleur conseil épargne 2024 (3) Prix du meilleur conseil épargne 2024 (3) Des conseillers experts en ligne et en agence Des conseillers experts en ligne et en agence Choisissez la carte qui vous convient Activation de la carte dès le premier paiement sans contact Vos plafonds de retrait et de paiement personnalisables Activation de la carte dès le premier paiement sans contact Activation de la cart

In [9]:
# Save as JSON (good for vector databases)
json_path = Path(OUTPUT_DIR) / "rag_documents.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)
print(f"✓ Saved JSON: {json_path}")

✓ Saved JSON: bnp_rag_documents/rag_documents.json


In [10]:
# Save as JSONL (one document per line - common RAG format)
jsonl_path = Path(OUTPUT_DIR) / "rag_documents.jsonl"
with open(jsonl_path, 'w', encoding='utf-8') as f:
    for doc in documents:
        f.write(json.dumps(doc, ensure_ascii=False) + '\n')
print(f"✓ Saved JSONL: {jsonl_path}")

✓ Saved JSONL: bnp_rag_documents/rag_documents.jsonl


In [11]:
# Create summary DataFrame
summary_df = pd.DataFrame([{
    'id': doc['id'],
    'category': doc['category'],
    'title': doc['title'],
    'url': doc['url'],
    'word_count': doc['word_count'],
    'has_error': 'error' in doc
} for doc in documents])

# Save summary as CSV
csv_path = Path(OUTPUT_DIR) / "scraping_summary.csv"
summary_df.to_csv(csv_path, index=False, encoding='utf-8')
print(f"✓ Saved summary: {csv_path}\n")

print("📄 Scraping Summary:")
display(summary_df)

✓ Saved summary: bnp_rag_documents/scraping_summary.csv

📄 Scraping Summary:


,id,category,title,url,word_count,has_error
0,doc_0,Accueil / Vue d’ensemble,BNP PARIBAS BANQUE ET ASSURANCES,https://mabanque.bnpparibas/,1865,False
1,doc_1,Comptes & Formules,Comptes & Formules,https://mabanque.bnpparibas/fr/devenir-client/...,0,True
2,doc_2,Épargne & Livrets,Épargne & Livrets,https://mabanque.bnpparibas/fr/epargner/compte...,170,False
3,doc_3,Crédits & Financement,Crédits & Financement,https://mabanque.bnpparibas/fr/vos-besoins/vos...,14,False
4,doc_4,Cartes bancaires,Cartes bancaires,https://mabanque.bnpparibas/fr/gerer/services-...,0,True
5,doc_5,Bourse & Placement,Nos offres et Services d’investissement en Bourse,https://mabanque.bnpparibas/fr/bourse/nos-offr...,567,False
6,doc_7,FAQ / Aide & Services,FAQ / Aide & Services,https://mabanque.bnpparibas/fr/ma-banque-et-mo...,2449,False
7,doc_8,Tarifs & Conditions,Tarifs & Conditions,https://mabanque.bnpparibas/fr/notre-offre/tarifs,0,True
8,doc_9,Sécurité / Connexion,Sécurité / Connexion,https://mabanque.bnpparibas/fr/securite,0,True


In [12]:
# Calculate statistics
total_words = sum(doc['word_count'] for doc in documents)
errors = sum(1 for doc in documents if 'error' in doc)
success_rate = ((len(documents)-errors)/len(documents)*100)

print(f"\n{'='*60}")
print("📊 STATISTICS")
print(f"{'='*60}")
print(f"  Total documents: {len(documents)}")
print(f"  Total words: {total_words:,}")
print(f"  Errors: {errors}")
print(f"  Success rate: {success_rate:.1f}%")
print(f"{'='*60}\n")

# Create stats DataFrame
stats_df = pd.DataFrame([{
    'Total documents': len(documents),
    'Total words': f"{total_words:,}",
    'Errors': errors,
    'Success rate': f"{success_rate:.1f}%"
}])
display(stats_df)


📊 STATISTICS
  Total documents: 9
  Total words: 5,065
  Errors: 4
  Success rate: 55.6%



,Total documents,Total words,Errors,Success rate
0,9,"5,065",4,55.6%


In [13]:
# Preview first 3 documents
print("📖 Preview of scraped documents:\n")
for i, doc in enumerate(documents[:3]):
    print(f"{'='*60}")
    print(f"Document {i+1}: {doc['category']}")
    print(f"{'='*60}")
    print(f"Title: {doc['title']}")
    print(f"URL: {doc['url']}")
    print(f"Word Count: {doc['word_count']}")
    print(f"\nContent Preview (first 500 chars):")
    print(doc['content'][:500] + "...")
    print(f"\n")

📖 Preview of scraped documents:

Document 1: Accueil / Vue d’ensemble
Title: BNP PARIBAS BANQUE ET ASSURANCES
URL: https://mabanque.bnpparibas/
Word Count: 1865

Content Preview (first 500 chars):
BNP PARIBAS BANQUE ET ASSURANCES Jusqu’à 270 € d’avantages offerts (1) Pour toute première ouverture d'un compte BNP Paribas et d'une carte Visa Premier avec l'offre Esprit Libre Référence Soit 80 € offerts + 1 an de gratuité aux services essentiels Esprit Libre. Leader européen des services bancaires (2) Leader européen des services bancaires (2) Prix du meilleur conseil épargne 2024 (3) Prix du meilleur conseil épargne 2024 (3) Des conseillers experts en ligne et en agence Des conseillers expe...


Document 2: Comptes & Formules
Title: Comptes & Formules
URL: https://mabanque.bnpparibas/fr/devenir-client/compte-bancaire
Word Count: 0

Content Preview (first 500 chars):
Error: Could not scrape this page. 404 Client Error: Not Found for url: https://mabanque.bnpparibas/fr/devenir-client/compt

In [14]:
# Create detailed summary DataFrame
summary_detailed = pd.DataFrame([{
    'ID': doc['id'],
    'Category': doc['category'],
    'Title': doc['title'],
    'Word Count': doc['word_count'],
    'Has Error': 'error' in doc,
    'Scraped At': doc['scraped_at']
} for doc in documents])

print("📊 Detailed Summary:")
display(summary_detailed)

📊 Detailed Summary:


,ID,Category,Title,Word Count,Has Error,Scraped At
0,doc_0,Accueil / Vue d’ensemble,BNP PARIBAS BANQUE ET ASSURANCES,1865,False,2025-10-19 22:47:48
1,doc_1,Comptes & Formules,Comptes & Formules,0,True,2025-10-19 22:47:51
2,doc_2,Épargne & Livrets,Épargne & Livrets,170,False,2025-10-19 22:47:54
3,doc_3,Crédits & Financement,Crédits & Financement,14,False,2025-10-19 22:47:56
4,doc_4,Cartes bancaires,Cartes bancaires,0,True,2025-10-19 22:47:59
5,doc_5,Bourse & Placement,Nos offres et Services d’investissement en Bourse,567,False,2025-10-19 22:48:02
6,doc_7,FAQ / Aide & Services,FAQ / Aide & Services,2449,False,2025-10-19 22:48:04
7,doc_8,Tarifs & Conditions,Tarifs & Conditions,0,True,2025-10-19 22:48:07
8,doc_9,Sécurité / Connexion,Sécurité / Connexion,0,True,2025-10-19 22:48:09
